In [1]:
from poker_dataset import PokerDataset
from poker_test import get_models_accuracy
from poker_baseline_model import baseline_model
from poker_model import CustomFCNetwork
import torch

In [2]:
filepath = r'C:\Users\kctak\Documents\Code\PokerAI\APS360 Progress\poker_data'
poker_dataset = PokerDataset(filepath, "IlxxxlI", balance_data=True)
print({action_name: len(indices) for action_name, indices in poker_dataset.output_indices.items()})

Processing Items: 100%|██████████| 50664/50664 [00:35<00:00, 1440.72it/s]

{'calls': 39175, 'raises': 39175, 'bets': 0, 'folds': 39175, 'checks': 39175, 'allin': 39175}


In [3]:
import torch.utils

test_frac = 0.15
val_frac = 0.15
train_frac = 1 - test_frac - val_frac

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(poker_dataset, [train_frac, val_frac, test_frac])

In [4]:
# Load trained model 
hidden_shape = [256, 128]
trained_model_path = r'Results2\poker_model.pth'

model = CustomFCNetwork(hidden_shape=hidden_shape)
model.load_state_dict(torch.load(trained_model_path))

<All keys matched successfully>

In [5]:
model_tracker, baseline_tracker = get_models_accuracy(model, baseline_model, test_dataset)

In [7]:
# model_tracker = {action_name: {'correct': [], 'wrong': []} for action_name in ACTION_NAMES}
# baseline_tracker = {action_name: {'correct': [], 'wrong': []} for action_name in ACTION_NAMES}

model_corr_count = {action_name: {keys: len(values) for keys, values in records.items()} for action_name, records in model_tracker.items() if not all([len(values)==0 for keys, values in records.items()])}
baseline_corr_count = {action_name: {keys: len(values) for keys, values in records.items()} for action_name, records in baseline_tracker.items() if not all([len(values)==0 for keys, values in records.items()])}

print("Model Correct Count", model_corr_count)
print("Baseline Correct Count", baseline_corr_count)

Model Correct Count {'calls': {'correct': 5900, 'wrong': 47}, 'raises': {'correct': 5792, 'wrong': 27}, 'folds': {'correct': 5737, 'wrong': 23}, 'checks': {'correct': 5964, 'wrong': 4}, 'allin': {'correct': 5887, 'wrong': 0}}
Baseline Correct Count {'calls': {'correct': 1766, 'wrong': 2766}, 'folds': {'correct': 5612, 'wrong': 17352}, 'allin': {'correct': 974, 'wrong': 911}}
